In [ ]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce

from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_squared_error

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [ ]:
train0 = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
train0

In [ ]:
test0 = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")
test0

In [ ]:
print(train0.columns.to_list())

In [ ]:
train_col=list(set(train0.columns.to_list())-set(['id','loss']))
target=['loss']
print(train_col)

In [ ]:
X = train0.drop(['id','loss'],axis=1)
y0 = train0['loss']
y1 = y0.apply(lambda x:np.log1p(x))
y = y1.apply(lambda x:np.log1p(x))
test = test0.drop('id',axis=1)

In [ ]:
fig,ax = plt.subplots(figsize=(16,4))
sns.histplot(y0, label='Train', ax=ax, color='C1',bins=43)
ax.legend()
ax.grid()

In [ ]:
fig,ax = plt.subplots(figsize=(16,4))
sns.histplot(y1, label='Train', ax=ax, color='C1',bins=43)
ax.legend()
ax.grid()

In [ ]:
fig,ax = plt.subplots(figsize=(16,4))
sns.histplot(y, label='Train', ax=ax, color='C1',bins=43)
ax.legend()
ax.grid()

In [ ]:
X

In [ ]:
test

In [ ]:
df_columns=train_col

In [ ]:
def create_numeric_feature(input_df):
    use_columns = df_columns 
    return input_df[use_columns].copy()

In [ ]:
from contextlib import contextmanager
from time import time

class Timer:
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' '):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [ ]:
from tqdm import tqdm

def to_feature(input_df):

    processors = [
        create_numeric_feature,
    ]
    
    out_df = pd.DataFrame()
    
    for func in tqdm(processors, total=len(processors)):
        with Timer(prefix='create' + func.__name__ + ' '):
            _df = func(input_df)

        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
        
    return out_df

In [ ]:
train_feat_df = to_feature(X)
test_feat_df = to_feature(test)

In [ ]:
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error

def fit_lgbm(X, y, cv, 
             params: dict=None, 
             verbose: int=50):

    if params is None:
        params = {}

    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        clf = lgbm.LGBMRegressor(**params)
        
        with Timer(prefix='fit fold={} '.format(i)):
            clf.fit(x_train, y_train, 
                    eval_set=[(x_valid, y_valid)],  
                    early_stopping_rounds=100,
                    verbose=verbose)

        pred_i = clf.predict(x_valid)
        oof_pred[idx_valid] = pred_i
        models.append(clf)
        print(f'Fold {i} RMSLE: {mean_squared_error(y_valid, pred_i) ** .5:.4f}')
        print()

    score = mean_squared_error(y, oof_pred) ** .5
    print('-' * 50)
    print('FINISHED | Whole RMSLE: {:.4f}'.format(score))
    return oof_pred, models

In [ ]:
params = {
 'num_leaves': 65, 
 'objective': 'rmse', 
 'max_depth': 4, 
 'learning_rate': 0.24576598352003948, 
 'lambda_l1': 8.201664065055316e-08, 
 'lambda_l2': 0.0046015787738683915, 
 'bagging_fraction': 0.6812701053947862, 
 'feature_fraction': 0.9459127652755307,
}

In [ ]:
from sklearn.model_selection import KFold
for i in range(1):
    fold = KFold(n_splits=5, shuffle=True, random_state=71)
    ydfi=y
    yar=np.array(ydfi)
    cv = list(fold.split(train_feat_df, yar))
    oof, models = fit_lgbm(train_feat_df.values, yar, cv, params=params, verbose=500)
    
    fig,ax = plt.subplots(figsize=(6,6))
    ax.set_title(target[i],fontsize=20)
    ax.set_xlabel('oof '+target[i],fontsize=12)
    ax.set_ylabel('train_y '+target[i],fontsize=12)
    ax.scatter(oof,yar)

In [ ]:
def visualize_importance(models, feat_train_df):

    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importances_
        _df['column'] = feat_train_df.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], 
                                          axis=0, ignore_index=True)

    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(8, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df, 
                  x='feature_importance', 
                  y='column', 
                  order=order, 
                  ax=ax, 
                  palette='viridis', 
                  orient='h')
    ax.tick_params(axis='x', rotation=0)
    #ax.set_title('Importance')
    ax.grid()
    fig.tight_layout()
    return fig,ax

#fig, ax = visualize_importance(models, train_feat_df)

In [ ]:
MODELS=[]
for i in range(1):
    fold = KFold(n_splits=5, shuffle=True, random_state=71)
    ydfi=y
    yar=np.array(ydfi)
    cv = list(fold.split(train_feat_df, yar))
    oof, models = fit_lgbm(train_feat_df.values, yar, cv, params=params, verbose=500)
    fig, ax = visualize_importance(models, train_feat_df)
    ax.set_title(target[i]+' Imortance',fontsize=20)
    MODELS+=[models]

In [ ]:
sample = pd.read_csv("../input/tabular-playground-series-aug-2021/sample_submission.csv")
sample

In [ ]:
MODELS

In [ ]:
subm=sample
for i in range(1):
    pred0=MODELS[i][4].predict(test_feat_df.values)
    pred1=pd.Series(pred0).apply(lambda x:np.expm1(x))  
    pred2=pd.Series(pred1).apply(lambda x:np.expm1(x))
    PRED=pd.Series(pred2).apply(lambda x:int(x*2))
    subm[target[i]]=PRED

In [ ]:
subm['loss'].value_counts()

In [ ]:
subm.to_csv('submission.csv',index=False)
subm

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
sns.histplot(PRED, label='Test Predict', ax=ax, color='black',bins=40)
sns.histplot(train0['loss'], label='Train', ax=ax, color='C1',bins=43)
ax.legend()
ax.grid()